In [2]:
import logging
from typing import List, Optional, Any, Dict, Tuple

import requests
import json
import warnings
import pandas as pd

from pydantic import BaseModel, create_model, validator, ValidationError
from langchain.chat_models.base import BaseChatModel
from langchain.schema import (
    BaseMessage,
    AIMessage,
    HumanMessage,
    SystemMessage,
    ChatResult,
    ChatGeneration,
)
from langchain.tools import StructuredTool
from langchain.agents import create_structured_chat_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from pydantic import PrivateAttr

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)

In [3]:
class Llama31ChatModel(BaseChatModel):
    api_key: str
    base_url: str
    model: str
    temperature: float = 0.5
    max_tokens: int = 3000
    logging_level: int = logging.INFO

    _logger: logging.Logger = PrivateAttr()

    class Config:
        arbitrary_types_allowed = True

    def __init__(self, **data):
        super().__init__(**data)
        self._logger = logging.getLogger(__name__)
        logging.basicConfig(level=self.logging_level)

    @property
    def _llm_type(self) -> str:
        return "llama31"

    def _prepare_headers(self) -> Dict[str, str]:
        return {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }

    def _prepare_context(self, messages: List[BaseMessage]) -> List[Dict[str, str]]:
        role_map = {
            HumanMessage: "user",
            AIMessage: "assistant",
            SystemMessage: "system",
        }
        return [
            {"role": role_map.get(type(message), "user"), "content": message.content}
            for message in messages
        ]

    def _prepare_payload(
        self,
        context: List[Dict[str, str]],
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> Dict[str, Any]:
        payload = {
            "model": self.model,
            "messages": context,
            "temperature": kwargs.get("temperature", self.temperature),
            "max_tokens": kwargs.get("max_tokens", self.max_tokens),
        }
        if stop is not None:
            payload["stop"] = stop
        return payload

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> ChatResult:
        headers = self._prepare_headers()
        context = self._prepare_context(messages)
        payload = self._prepare_payload(context, stop, **kwargs)

        try:
            response = requests.post(
                f"{self.base_url}/chat/completions",
                headers=headers,
                json=payload,
            )
            response.raise_for_status()
            assistant_response = response.json()["choices"][0]["message"]["content"]
            self._logger.info("REQUEST: %s", payload)
            self._logger.info("RESPONSE: %s", assistant_response)
            ai_message = AIMessage(content=assistant_response)
            generation = ChatGeneration(message=ai_message)
            return ChatResult(generations=[generation])
        except requests.RequestException as e:
            self._logger.error("API request failed: %s", e)
            raise RuntimeError(f"API request failed: {e}")

In [26]:
system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a JSON blob to specify a tool by providing an "action" key (tool name) and an "action_input" key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per JSON blob, as shown:

{{ "action": $TOOL_NAME, "action_input": $INPUT }}

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action: $JSON_BLOB

Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action: {{ "action": "Final Answer", "action_input": "Final response to human" }}

Begin! Reminder to ALWAYS respond with a valid JSON blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB``` then Observation
Return a list of Actions to answer all questions in request. Use multiple tools to answer one question!
Do NOT include any text outside the JSON.
If you want to provide your thoughts, do not output them.
Output only the JSON with one "action" and one "action_input".
"""

human_prompt = """{input}
{agent_scratchpad}
(Reminder to respond in a JSON blob no matter what)"""

system_message = SystemMessagePromptTemplate.from_template(
    system_prompt,
    input_variables=["tools", "tool_names"],
)
human_message = HumanMessagePromptTemplate.from_template(
    human_prompt,
    input_variables=["input", "agent_scratchpad"],
)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message,
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        human_message,
    ]
)

In [11]:
def parse_type(type_str: str):
    parts = [p.strip() for p in type_str.split(",")]
    base = parts[0]

    is_optional = False
    if len(parts) > 1 and "optional" in parts[1:]:
        is_optional = True

    if base == "str":
        py_type = str
    elif base == "int":
        py_type = int
    elif base == "bool":
        py_type = bool
    elif base == "Dict":
        py_type = Dict[Any, Any]
    elif base == "List":
        py_type = List[Any]
    elif base == "List[float]":
        py_type = List[float]
    elif base == "List[int]":
        py_type = List[int]
    elif base == "Tuple[float, float]":
        py_type = Tuple[float, float]
    else:
        py_type = Any

    if is_optional:
        return (Optional[py_type], None)
    else:
        return (py_type, ...)

In [12]:
def create_tool_input_model(func_name: str, parameters: Dict[str, Any]) -> type:
    field_definitions = {}
    for param_name, param_type in parameters.items():
        field_definitions[param_name] = param_type

    class ConfiguredModel(BaseModel):
        @validator("*", pre=True)
        def parse_string_list(cls, v):
            if (
                isinstance(v, str)
                and v.strip().startswith("[")
                and v.strip().endswith("]")
            ):
                try:
                    loaded = json.loads(v)
                    if isinstance(loaded, list):
                        return loaded
                except json.JSONDecodeError:
                    pass
            return v

    dynamic_model = create_model(
        f"{func_name}Input", __base__=ConfiguredModel, **field_definitions
    )

    return dynamic_model

In [13]:
def create_tool_function(
    func_name: str,
    func_description: str,
    parameters: Dict[str, Any],
):
    InputModel = create_tool_input_model(func_name, parameters)

    def simulated_api(**kwargs):
        return f"Simulated response for {func_name}, input={kwargs}"

    return StructuredTool.from_function(
        func=simulated_api,
        name=func_name,
        description=func_description,
        args_schema=InputModel,
        return_direct=True,
    )

In [8]:
df = pd.read_json(
    "hf://datasets/Salesforce/xlam-function-calling-60k/xlam_function_calling_60k.json"
)
ids = [
    47119,
    43159,
    52763,
    6431,
    428,
    40876,
    36526,
    37040,
    46776,
    48062,
    51904,
    39364,
    39885,
    33614,
    28877,
    3798,
    39001,
    38624,
    46181,
    28392,
    50940,
]
df = df.iloc[ids]
df["query"] = df["query"].astype(str)
df["answers"] = df["answers"].apply(lambda x: json.loads(x))
df["tools"] = df["tools"].apply(lambda x: json.loads(x))
df

,id,query,answers,tools
47119,47119,"Find the latest 20 tweets from the user with the ID 456789123 and get the Instagram user information for the user with the ID 111222333. Then, download the song from this SoundCloud URL: 'https://soundcloud.com/user/other_song' and retrieve information for the Instagram user with the username 'jane_doe'. Finally, get all images and videos from the Instagram highlight with the ID 'ABC789'.","[{'name': 'get_user_tweets', 'arguments': {'count': '20', 'user': '456789123'}}, {'name': 'get_user_info_by_user_id', 'arguments': {'is_id': 111222333}}, {'name': 'song_download', 'arguments': {'track_url': 'https://soundcloud.com/user/other_song'}}, {'name': 'info', 'arguments': {'username': 'jane_doe'}}, {'name': 'get_highlight_by_id', 'arguments': {'highlightid': 'ABC789'}}]","[{'name': 'song_download', 'description': 'Downloads a song from the provided SoundCloud URL using the specified RapidAPI key.', 'parameters': {'track_url': {'description': 'The URL of the SoundCloud track to be downloaded.', 'type': 'str', 'default': 'https://soundcloud.com/user-977421934/the-phoenix'}}}, {'name': 'get_user_info_by_user_id', 'description': 'Fetch Instagram user information by user ID using the RapidAPI service.', 'parameters': {'is_id': {'description': 'The Instagram user ID for which the information is to be fetched.', 'type': 'int', 'default': '18527'}}}, {'name': 'get_user_tweets', 'description': 'Fetches tweets for a specified user using the provided RapidAPI key.', 'parameters': {'count': {'description': 'The number of tweets to retrieve.', 'type': 'str', 'default': '20'}, 'user': {'description': 'The ID of the user whose tweets are to be fetched.', 'type': 'str', 'default': '2455740283'}, 'cursor': {'description': 'The cursor for pagination. Default is None.', 'type': 'str, optional', 'default': ''}}}, {'name': 'info', 'description': 'Fetches profile information for a given Instagram username using the RapidAPI service.', 'parameters': {'username': {'description': ""The Instagram username for which to fetch profile information. Defaults to 'kimkardashian'."", 'type': 'str', 'default': 'kimkardashian'}}}, {'name': 'get_highlight_by_id', 'description': 'Fetches all images and videos of a specific Instagram highlight by its ID.', 'parameters': {'highlightid': {'description': 'The ID of the Instagram highlight.', 'type': 'str', 'default': 'highlight:18042967741398634'}}}]"
43159,43159,"Find a pastry recipe with 'chocolate' as an ingredient, get the drink recipe with ID '123', and fetch the recipe with ID '456' from the Ghana Food Recipe API. Also, generate a recipe using 'beef' as the main ingredient and get the menu details for ID '789' in English.","[{'name': 'pastry_ingredient', 'arguments': {'ingredient': 'chocolate'}}, {'name': 'drink_by_id', 'arguments': {'is_id': '123'}}, {'name': 'get_recipe_by_id', 'arguments': {'is_id': '456'}}, {'name': 'generate_recipe', 'arguments': {'ingredient': 'beef'}}, {'name': 'menudetails', 'arguments': {'is_id': '789', 'lang': 'en'}}]","[{'name': 'pastry_ingredient', 'description': 'Retrieves a random pastry recipe that contains a specific ingredient using the Toolbench RapidAPI.', 'parameters': {'ingredient': {'description': 'The specific ingredient to include in the pastry recipe.', 'type': 'str', 'default': 'strawberry'}}}, {'name': 'drink_by_id', 'description': 'Fetches a specific drink recipe by its ID using the Ghana Food Recipe API.', 'parameters': {'is_id': {'description': 'The unique identifier for the drink recipe.', 'type': 'str', 'default': '63c67c9db06a3ed9f92b2367'}}}, {'name': 'get_recipe_by_id', 'description': 'Fetches a specific recipe by its ID from the Ghana Food Recipe API.', 'parameters': {'is_id': {'description': 'The ID of the recipe to fetch.', 'type': 'str', 'default': '63c2e83b09852be5facb894a'}}}, {'name': 'generate_recipe', 'description': 'This function generates a recipe using an ingredient and a provided RapidAPI key.', 'parameters': {'ingred

In [29]:
for idx, row in df.iterrows():
    tools_data = row["tools"]
    all_created_tools = []

    for tool_def in tools_data:
        func_name = tool_def["name"]
        func_description = tool_def["description"]
        parameters_info = tool_def["parameters"]

        pydantic_params = {}
        for param_name, param_meta in parameters_info.items():
            param_type_str = param_meta["type"]
            pydantic_params[param_name] = parse_type(param_type_str)

        tool = create_tool_function(
            func_name=func_name,
            func_description=func_description,
            parameters=pydantic_params,
        )
        all_created_tools.append(tool)

    llm = Llama31ChatModel(
        api_key="sk-or-vv-",
        base_url="https://api.vsegpt.ru/v1",
        model="meta-llama/llama-3.3-70b-instruct",
        temperature=0.5,
        max_tokens=3000,
    )

    agent = create_structured_chat_agent(
        llm=llm,
        tools=all_created_tools,
        prompt=prompt,
        stop_sequence=True,
    )

    agent_executor = AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=all_created_tools,
        verbose=True,
        return_intermediate_steps=True,
        output_keys=["output"],
        handle_parsing_errors=True,
    )

    simulation_input = {
        "input": row["query"],
        "agent_scratchpad": "",
    }

    result = agent_executor(simulation_input)
    print(f"ID: {idx}")
    print()
    print("=========== РЕЗУЛЬТАТ СИМУЛЯЦИИ ===========")
    print(result)
    print("===========================================")



> Entering new AgentExecutor chain...


INFO:__main__:REQUEST: {'model': 'meta-llama/llama-3.3-70b-instruct', 'messages': [{'role': 'system', 'content': 'Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nsong_download(**kwargs) - Downloads a song from the provided SoundCloud URL using the specified RapidAPI key., args: {\'track_url\': {\'title\': \'Track Url\', \'type\': \'string\'}}\nget_user_info_by_user_id(**kwargs) - Fetch Instagram user information by user ID using the RapidAPI service., args: {\'is_id\': {\'title\': \'Is Id\', \'type\': \'integer\'}}\nget_user_tweets(**kwargs) - Fetches tweets for a specified user using the provided RapidAPI key., args: {\'count\': {\'title\': \'Count\', \'type\': \'string\'}, \'user\': {\'title\': \'User\', \'type\': \'string\'}, \'cursor\': {\'anyOf\': [{\'type\': \'string\'}, {\'type\': \'null\'}], \'default\': None, \'title\': \'Cursor\'}}\ninfo(**kwargs) - Fetches profile information for a given Instagram username using the Ra

{"action": "get_user_tweets", "action_input": {"user": "456789123", "count": "20", "cursor": "null"}} 
{"action": "get_user_info_by_user_id", "action_input": {"is_id": "111222333"}} 
{"action": "song_download", "action_input": {"track_url": "https://soundcloud.com/user/other_song"}} 
{"action": "info", "action_input": {"username": "jane_doe"}} 
{"action": "get_highlight_by_id", "action_input": {"highlightid": "ABC789"}}Simulated response for get_user_tweets, input={'count': '20', 'user': '456789123', 'cursor': 'null'}


> Finished chain.
ID: 47119

=========== РЕЗУЛЬТАТ СИМУЛЯЦИИ ===========
{'input': "Find the latest 20 tweets from the user with the ID 456789123 and get the Instagram user information for the user with the ID 111222333. Then, download the song from this SoundCloud URL: 'https://soundcloud.com/user/other_song' and retrieve information for the Instagram user with the username 'jane_doe'. Finally, get all images and videos from the Instagram highlight with the ID 'ABC789'

INFO:__main__:REQUEST: {'model': 'meta-llama/llama-3.3-70b-instruct', 'messages': [{'role': 'system', 'content': 'Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\npastry_ingredient(**kwargs) - Retrieves a random pastry recipe that contains a specific ingredient using the Toolbench RapidAPI., args: {\'ingredient\': {\'title\': \'Ingredient\', \'type\': \'string\'}}\ndrink_by_id(**kwargs) - Fetches a specific drink recipe by its ID using the Ghana Food Recipe API., args: {\'is_id\': {\'title\': \'Is Id\', \'type\': \'string\'}}\nget_recipe_by_id(**kwargs) - Fetches a specific recipe by its ID from the Ghana Food Recipe API., args: {\'is_id\': {\'title\': \'Is Id\', \'type\': \'string\'}}\ngenerate_recipe(**kwargs) - This function generates a recipe using an ingredient and a provided RapidAPI key., args: {\'ingredient\': {\'title\': \'Ingredient\', \'type\': \'string\'}}\nmenudetails(**kwargs) - Fetch menu details using a given menu 

{"action": "pastry_ingredient", "action_input": {"ingredient": "chocolate"}}Simulated response for pastry_ingredient, input={'ingredient': 'chocolate'}


> Finished chain.
ID: 43159

=========== РЕЗУЛЬТАТ СИМУЛЯЦИИ ===========
{'input': "Find a pastry recipe with 'chocolate' as an ingredient, get the drink recipe with ID '123', and fetch the recipe with ID '456' from the Ghana Food Recipe API. Also, generate a recipe using 'beef' as the main ingredient and get the menu details for ID '789' in English.", 'agent_scratchpad': '', 'output': "Simulated response for pastry_ingredient, input={'ingredient': 'chocolate'}", 'intermediate_steps': [(AgentAction(tool='pastry_ingredient', tool_input={'ingredient': 'chocolate'}, log='{"action": "pastry_ingredient", "action_input": {"ingredient": "chocolate"}}'), "Simulated response for pastry_ingredient, input={'ingredient': 'chocolate'}")]}


> Entering new AgentExecutor chain...


...

{"action": "listing_status", "action_input": {"is_id": "66666", "year": 2029, "month": 8}} 
{"action": "biggest_cities", "action_input": {"is_id": 17}} 
{"action": "route_information", "action_input": {"cmd": "get_station_accessibility"}} 
{"action": "get_host_language_filters", "action_input": {"languageid": "pt-PT"}} 
{"action": "list_of_room_facility_types", "action_input": {"facility_type_id": "120"}}Simulated response for listing_status, input={'is_id': '66666', 'year': 2029, 'month': 8}


> Finished chain.
ID: 46181

=========== РЕЗУЛЬТАТ СИМУЛЯЦИИ ===========
{'input': "1. Check the availability status of a listing with ID '66666' for August 2029. 2. Fetch data for the biggest city with ID '17'. 3. Obtain route information for the BART API with command 'get_station_accessibility'. 4. Get host language filters using language ID 'pt-PT'. 5. Fetch a list of room facility types for facility type ID '120'.", 'agent_scratchpad': '', 'output': "Simulated response for listing_status, in

INFO:__main__:REQUEST: {'model': 'meta-llama/llama-3.3-70b-instruct', 'messages': [{'role': 'system', 'content': 'Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nfibonacci(**kwargs) - Calculates the nth Fibonacci number., args: {\'n\': {\'title\': \'N\', \'type\': \'integer\'}}\ncalculate_factorial(**kwargs) - Calculates the factorial of a non-negative integer., args: {\'n\': {\'title\': \'N\', \'type\': \'integer\'}}\nfind_max_subarray_sum(**kwargs) - Finds the maximum sum of a contiguous subarray within a list of integers., args: {\'nums\': {\'items\': {\'type\': \'integer\'}, \'title\': \'Nums\', \'type\': \'array\'}}\nreverse_words(**kwargs) - Reverses the order of words in a sentence., args: {\'sentence\': {\'title\': \'Sentence\', \'type\': \'string\'}}\nreverse_string(**kwargs) - Reverses the characters in a string., args: {\'text\': {\'title\': \'Text\', \'type\': \'string\'}}\ncalculate_grade(**kwargs) - Calculates the w

{"action": "fibonacci", "action_input": {"n": 10}}Simulated response for fibonacci, input={'n': 10}


> Finished chain.
ID: 28392

=========== РЕЗУЛЬТАТ СИМУЛЯЦИИ ===========
{'input': "I need to find the 10th Fibonacci number, the factorial of 7, and the maximum subarray sum of the list [1, -2, 3, 10, -8, 9, 2]. Also, could you reverse the words in the sentence 'Hello world this is a test' and the characters in the string 'Python'?", 'agent_scratchpad': '', 'output': "Simulated response for fibonacci, input={'n': 10}", 'intermediate_steps': [(AgentAction(tool='fibonacci', tool_input={'n': 10}, log='{"action": "fibonacci", "action_input": {"n": 10}}'), "Simulated response for fibonacci, input={'n': 10}")]}


> Entering new AgentExecutor chain...


INFO:__main__:REQUEST: {'model': 'meta-llama/llama-3.3-70b-instruct', 'messages': [{'role': 'system', 'content': 'Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nlisting_status(**kwargs) - Retrieve the availability status of a listing for a given month and year., args: {\'is_id\': {\'title\': \'Is Id\', \'type\': \'string\'}, \'year\': {\'title\': \'Year\', \'type\': \'integer\'}, \'month\': {\'title\': \'Month\', \'type\': \'integer\'}}\nbiggest_cities(**kwargs) - Fetches the biggest cities\' data from the specified API using the provided ID and API key., args: {\'is_id\': {\'title\': \'Is Id\', \'type\': \'integer\'}}\nroute_information(**kwargs) - Fetch route information from the BART API using the given command., args: {\'cmd\': {\'title\': \'Cmd\', \'type\': \'string\'}}\nget_host_language_filters(**kwargs) - Fetches host language filters from the Airbnb API., args: {\'languageid\': {\'anyOf\': [{\'type\': \'string\'}, {\'ty

{"action": "listing_status", "action_input": {"is_id": "67890", "year": 2023, "month": 3}} 
{"action": "biggest_cities", "action_input": {"is_id": 5}} 
{"action": "route_information", "action_input": {"cmd": "get_station_details"}} 
{"action": "get_host_language_filters", "action_input": {"languageid": "es-ES"}} 
{"action": "list_of_room_facility_types", "action_input": {"facility_type_id": "30"}}Simulated response for listing_status, input={'is_id': '67890', 'year': 2023, 'month': 3}


> Finished chain.
ID: 50940

=========== РЕЗУЛЬТАТ СИМУЛЯЦИИ ===========
{'input': "1. Check the availability status of a listing with ID '67890' for March 2023. 2. Fetch data for the biggest city with ID '5'. 3. Obtain route information for the BART API with command 'get_station_details'. 4. Get host language filters using language ID 'es-ES'. 5. Fetch a list of room facility types for facility type ID '30'.", 'agent_scratchpad': '', 'output': "Simulated response for listing_status, input={'is_id': '67

In [30]:
df.to_json(
    "/Users/galyukshev/Desktop/smol-llm-research/xlam-function-calling-60k/xlam-function-calling-60k-used.json",
    orient="records",
    indent=4,
)